In [33]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit
from numpy import random
import matplotlib.pyplot as plt
from keras import Sequential, Model
from keras.backend import tensorflow_backend as K
from keras.layers import LSTM, Dense, Conv1D
import datetime

In [2]:
df = pd.read_csv('data_full_noTemp.csv', parse_dates=['timestamp'], index_col=0)

/Users/kategroschner/anaconda/lib/python3.6/site-packages/numpy/lib/arraysetops.py:472: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask |= (ar1 == a)


In [3]:
def stratified_split(df, features):
    '''
    df: the input dataframe to be split
    features: features to be used in stratified splitting
    return train-test split using stratified splitting
    '''
    p = 0.2 #portion of test set
    sss = StratifiedShuffleSplit(n_splits=1, test_size=p)
    building_profiles = (df.groupby('building_name').first())[features]
    X = df['building_name'].unique()
    y = building_profiles.reset_index(drop=True)
    for train_index, test_index in sss.split(X, y):
        train_buildings = X[train_index]
        test_buildings = X[test_index]
    train = df[df['building_name'].isin(train_buildings)]
    test = df[df['building_name'].isin(test_buildings)]
    return train, test

In [5]:
features = ['industry']
train, test = stratified_split(df, features)

In [15]:
train.to_csv('train_full_noTemp.csv')
test.to_csv('test_full_noTemp.csv')

In [72]:
def monthsplit(df,months,lastidx):
    X = []
    Y = []
    for n in df['building_name'].drop_duplicates():
        X_n = []
        Y_n = []
        subset = df[df['building_name']==n]
        for row in subset.iterrows():
            row = np.array(row[-1])
            if row[8] in months:
                r = np.concatenate((row[1:2],row[5:6],row[11:lastidx]))
                X_n.append(r)
            else:
                Y_n.append(row[1])
        X.append(X_n)
        Y.append(Y_n)
    X = np.array(X)
    Y = np.array(Y)
    return X,Y

In [73]:
trainX, trainY = monthsplit(train,[12,1,2],-1)

In [ ]:
testX, testY = monthsplit(test)

In [74]:
np.save('trainX_full_noTemp_wElec.npy',trainX)
np.save('trainY_full_noTemp_wElec.npy',trainY)

In [ ]:
def monthsplit_noFeat(df,months,lastidx):
    X = []
    Y = []
    for n in df['building_name'].drop_duplicates():
        X_n = []
        Y_n = []
        subset = df[df['building_name']==n]
        for row in subset.iterrows():
            row = np.array(row[-1])
            if row[8] in months:
                r = np.concatenate((row[5:6],row[11:lastidx]))
                X_n.append(r)
            else:
                Y_n.append(row[1])
        X.append(X_n)
        Y.append(Y_n)
    X = np.array(X)
    Y = np.array(Y)
    return X,Y

In [12]:
model = Sequential()
model.add(LSTM(256, input_shape=(2160,51)))
model.add(Dense(6600))
model.compile(loss='mean_squared_error', optimizer='adam')

In [17]:
buildings = df['building_name'].unique() #list of name of buildings
num_buildings = len(buildings)
num_obs = len(df) // num_buildings

In [18]:
num_obs

8760

In [75]:
def make_LSTM_data(df):
    buildings = df['building_name'].unique() #list of name of buildings
    num_buildings = len(buildings) # number of buildings
    num_obs = len(df) // num_buildings #number of observations per uilding
    three_months_hours = 24*90 #three months in hour
    nine_months_hours = num_obs - three_months_hours #nine month is defined to be the rest
    #make a long dataframe by duplicating
    df = pd.concat([df, df], ignore_index=True)

    X = pd.DataFrame() #data for fitting which contains three months data
    Y = pd.DataFrame() #data for evaluating which contains nine months data
    for building in buildings:
        subset = df[df['building_name']==building]
#         print(subset.shape)
#         display(subset.head())
#         display(subset.tail())
#         print(subset.columns)
#         break
        start_idx = np.random.randint(low=0, high=num_obs//24)
#         start_day = datetime.timedelta(np.random.randint(low=0, high=num_obs//24)) #randomly pick a startday
#         first_day = subset['timestamp'].iloc[0]
#         start_day = first_day + start_day
#         start_idx = subset[subset['timestamp']==start_day].index[0] #index of the start day
        x = subset.iloc[start_idx: start_idx+three_months_hours] #three months data
        X = pd.concat([X, x], axis=0)
        y = subset.iloc[start_idx+three_months_hours: start_idx+num_obs] #nine months data
        Y = pd.concat([Y, y], axis=0)
    return X, Y

In [64]:
pdTrainX,pdTrainY = make_LSTM_data(train)

,timestamp,electricity,building_name,weather_file,industry,area,primary_space_usage,year,month,hour,...,weekday_5,weekday_6,primary_space_usage_Office,primary_space_usage_PrimClass,primary_space_usage_UnivClass,primary_space_usage_UnivDorm,primary_space_usage_UnivLab,industry_Commercial Property,industry_Education,industry_Government
0,2015-01-01 07:00:00,27.79,Office_Abbey,weather0.csv,Education,6891.54454,Office,2015,1,7,...,0,0,1,0,0,0,0,0,1,0
1,2015-01-01 08:00:00,27.66,Office_Abbey,weather0.csv,Education,6891.54454,Office,2015,1,8,...,0,0,1,0,0,0,0,0,1,0
2,2015-01-01 09:00:00,27.13,Office_Abbey,weather0.csv,Education,6891.54454,Office,2015,1,9,...,0,0,1,0,0,0,0,0,1,0
3,2015-01-01 10:00:00,27.34,Office_Abbey,weather0.csv,Education,6891.54454,Office,2015,1,10,...,0,0,1,0,0,0,0,0,1,0
4,2015-01-01 11:00:00,27.34,Office_Abbey,weather0.csv,Education,6891.54454,Office,2015,1,11,...,0,0,1,0,0,0,0,0,1,0


,timestamp,electricity,building_name,weather_file,industry,area,primary_space_usage,year,month,hour,...,weekday_5,weekday_6,primary_space_usage_Office,primary_space_usage_PrimClass,primary_space_usage_UnivClass,primary_space_usage_UnivDorm,primary_space_usage_UnivLab,industry_Commercial Property,industry_Education,industry_Government
8760,2015-01-01 07:00:00,5.33,Office_Abigail,weather0.csv,Education,901.437809,Office,2015,1,7,...,0,0,1,0,0,0,0,0,1,0
8761,2015-01-01 08:00:00,5.41,Office_Abigail,weather0.csv,Education,901.437809,Office,2015,1,8,...,0,0,1,0,0,0,0,0,1,0
8762,2015-01-01 09:00:00,5.60,Office_Abigail,weather0.csv,Education,901.437809,Office,2015,1,9,...,0,0,1,0,0,0,0,0,1,0
8763,2015-01-01 10:00:00,6.50,Office_Abigail,weather0.csv,Education,901.437809,Office,2015,1,10,...,0,0,1,0,0,0,0,0,1,0
8764,2015-01-01 11:00:00,5.39,Office_Abigail,weather0.csv,Education,901.437809,Office,2015,1,11,...,0,0,1,0,0,0,0,0,1,0


,timestamp,electricity,building_name,weather_file,industry,area,primary_space_usage,year,month,hour,...,weekday_5,weekday_6,primary_space_usage_Office,primary_space_usage_PrimClass,primary_space_usage_UnivClass,primary_space_usage_UnivDorm,primary_space_usage_UnivLab,industry_Commercial Property,industry_Education,industry_Government
17520,2015-01-01 07:00:00,298.37,Office_Aliyah,weather0.csv,Education,18806.81881,Office,2015,1,7,...,0,0,1,0,0,0,0,0,1,0
17521,2015-01-01 08:00:00,294.33,Office_Aliyah,weather0.csv,Education,18806.81881,Office,2015,1,8,...,0,0,1,0,0,0,0,0,1,0
17522,2015-01-01 09:00:00,295.58,Office_Aliyah,weather0.csv,Education,18806.81881,Office,2015,1,9,...,0,0,1,0,0,0,0,0,1,0
17523,2015-01-01 10:00:00,294.79,Office_Aliyah,weather0.csv,Education,18806.81881,Office,2015,1,10,...,0,0,1,0,0,0,0,0,1,0
17524,2015-01-01 11:00:00,310.37,Office_Aliyah,weather0.csv,Education,18806.81881,Office,2015,1,11,...,0,0,1,0,0,0,0,0,1,0


,timestamp,electricity,building_name,weather_file,industry,area,primary_space_usage,year,month,hour,...,weekday_5,weekday_6,primary_space_usage_Office,primary_space_usage_PrimClass,primary_space_usage_UnivClass,primary_space_usage_UnivDorm,primary_space_usage_UnivLab,industry_Commercial Property,industry_Education,industry_Government
26280,2015-01-01 07:00:00,10.69,Office_Allyson,weather0.csv,Education,2670.96125,Office,2015,1,7,...,0,0,1,0,0,0,0,0,1,0
26281,2015-01-01 08:00:00,10.53,Office_Allyson,weather0.csv,Education,2670.96125,Office,2015,1,8,...,0,0,1,0,0,0,0,0,1,0
26282,2015-01-01 09:00:00,10.21,Office_Allyson,weather0.csv,Education,2670.96125,Office,2015,1,9,...,0,0,1,0,0,0,0,0,1,0
26283,2015-01-01 10:00:00,10.30,Office_Allyson,weather0.csv,Education,2670.96125,Office,2015,1,10,...,0,0,1,0,0,0,0,0,1,0
26284,2015-01-01 11:00:00,10.43,Office_Allyson,weather0.csv,Education,2670.96125,Office,2015,1,11,...,0,0,1,0,0,0,0,0,1,0


,timestamp,electricity,building_name,weather_file,industry,area,primary_space_usage,year,month,hour,...,weekday_5,weekday_6,primary_space_usage_Office,primary_space_usage_PrimClass,primary_space_usage_UnivClass,primary_space_usage_UnivDorm,primary_space_usage_UnivLab,industry_Commercial Property,industry_Education,industry_Government
35040,2015-01-01 07:00:00,286.450,Office_Alyson,weather0.csv,Education,14463.32485,Office,2015,1,7,...,0,0,1,0,0,0,0,0,1,0
35041,2015-01-01 08:00:00,276.415,Office_Alyson,weather0.csv,Education,14463.32485,Office,2015,1,8,...,0,0,1,0,0,0,0,0,1,0
35042,2015-01-01 09:00:00,264.730,Office_Alyson,weather0.csv,Education,14463.32485,Office,2015,1,9,...,0,0,1,0,0,0,0,0,1,0
35043,2015-01-01 10:00:00,261.830,Office_Alyson,weather0.csv,Education,14463.32485,Office,2015,1,10,...,0,0,1,0,0,0,0,0,1,0
35044,2015-01-01 11:00:00,267.535,Office_Alyson,weather0.csv,Education,14463.32485,Office,2015,1,11,...,0,0,1,0,0,0,0,0,1,0


...

,timestamp,electricity,building_name,weather_file,industry,area,primary_space_usage,year,month,hour,...,weekday_5,weekday_6,primary_space_usage_Office,primary_space_usage_PrimClass,primary_space_usage_UnivClass,primary_space_usage_UnivDorm,primary_space_usage_UnivLab,industry_Commercial Property,industry_Education,industry_Government
3504000,2014-12-01 00:00:00,146.0,UnivLab_Suzette,weather6.csv,Education,7810.0,UnivLab,2014,12,0,...,0,0,0,0,0,0,1,0,1,0
3504001,2014-12-01 01:00:00,144.0,UnivLab_Suzette,weather6.csv,Education,7810.0,UnivLab,2014,12,1,...,0,0,0,0,0,0,1,0,1,0
3504002,2014-12-01 02:00:00,142.0,UnivLab_Suzette,weather6.csv,Education,7810.0,UnivLab,2014,12,2,...,0,0,0,0,0,0,1,0,1,0
3504003,2014-12-01 03:00:00,137.0,UnivLab_Suzette,weather6.csv,Education,7810.0,UnivLab,2014,12,3,...,0,0,0,0,0,0,1,0,1,0
3504004,2014-12-01 04:00:00,138.0,UnivLab_Suzette,weather6.csv,Education,7810.0,UnivLab,2014,12,4,...,0,0,0,0,0,0,1,0,1,0


,timestamp,electricity,building_name,weather_file,industry,area,primary_space_usage,year,month,hour,...,weekday_5,weekday_6,primary_space_usage_Office,primary_space_usage_PrimClass,primary_space_usage_UnivClass,primary_space_usage_UnivDorm,primary_space_usage_UnivLab,industry_Commercial Property,industry_Education,industry_Government
3512760,2012-12-31 23:00:00,52.25,UnivLab_Tami,weather8.csv,Education,6873.1,UnivLab,2012,12,23,...,0,0,0,0,0,0,1,0,1,0
3512761,2013-01-01 00:00:00,52.75,UnivLab_Tami,weather8.csv,Education,6873.1,UnivLab,2013,1,0,...,0,0,0,0,0,0,1,0,1,0
3512762,2013-01-01 01:00:00,52.75,UnivLab_Tami,weather8.csv,Education,6873.1,UnivLab,2013,1,1,...,0,0,0,0,0,0,1,0,1,0
3512763,2013-01-01 02:00:00,52.75,UnivLab_Tami,weather8.csv,Education,6873.1,UnivLab,2013,1,2,...,0,0,0,0,0,0,1,0,1,0
3512764,2013-01-01 03:00:00,53.50,UnivLab_Tami,weather8.csv,Education,6873.1,UnivLab,2013,1,3,...,0,0,0,0,0,0,1,0,1,0


,timestamp,electricity,building_name,weather_file,industry,area,primary_space_usage,year,month,hour,...,weekday_5,weekday_6,primary_space_usage_Office,primary_space_usage_PrimClass,primary_space_usage_UnivClass,primary_space_usage_UnivDorm,primary_space_usage_UnivLab,industry_Commercial Property,industry_Education,industry_Government
3521520,2012-12-31 23:00:00,43.13575,UnivLab_Taylor,weather8.csv,Education,6033.2,UnivLab,2012,12,23,...,0,0,0,0,0,0,1,0,1,0
3521521,2013-01-01 00:00:00,43.00400,UnivLab_Taylor,weather8.csv,Education,6033.2,UnivLab,2013,1,0,...,0,0,0,0,0,0,1,0,1,0
3521522,2013-01-01 01:00:00,43.27825,UnivLab_Taylor,weather8.csv,Education,6033.2,UnivLab,2013,1,1,...,0,0,0,0,0,0,1,0,1,0
3521523,2013-01-01 02:00:00,44.86325,UnivLab_Taylor,weather8.csv,Education,6033.2,UnivLab,2013,1,2,...,0,0,0,0,0,0,1,0,1,0
3521524,2013-01-01 03:00:00,42.73625,UnivLab_Taylor,weather8.csv,Education,6033.2,UnivLab,2013,1,3,...,0,0,0,0,0,0,1,0,1,0


,timestamp,electricity,building_name,weather_file,industry,area,primary_space_usage,year,month,hour,...,weekday_5,weekday_6,primary_space_usage_Office,primary_space_usage_PrimClass,primary_space_usage_UnivClass,primary_space_usage_UnivDorm,primary_space_usage_UnivLab,industry_Commercial Property,industry_Education,industry_Government
3530280,2012-12-31 23:00:00,62.71100,UnivLab_Terrie,weather8.csv,Education,8030.2,UnivLab,2012,12,23,...,0,0,0,0,0,0,1,0,1,0
3530281,2013-01-01 00:00:00,62.11750,UnivLab_Terrie,weather8.csv,Education,8030.2,UnivLab,2013,1,0,...,0,0,0,0,0,0,1,0,1,0
3530282,2013-01-01 01:00:00,62.97675,UnivLab_Terrie,weather8.csv,Education,8030.2,UnivLab,2013,1,1,...,0,0,0,0,0,0,1,0,1,0
3530283,2013-01-01 02:00:00,61.98875,UnivLab_Terrie,weather8.csv,Education,8030.2,UnivLab,2013,1,2,...,0,0,0,0,0,0,1,0,1,0
3530284,2013-01-01 03:00:00,62.71925,UnivLab_Terrie,weather8.csv,Education,8030.2,UnivLab,2013,1,3,...,0,0,0,0,0,0,1,0,1,0


,timestamp,electricity,building_name,weather_file,industry,area,primary_space_usage,year,month,hour,...,weekday_5,weekday_6,primary_space_usage_Office,primary_space_usage_PrimClass,primary_space_usage_UnivClass,primary_space_usage_UnivDorm,primary_space_usage_UnivLab,industry_Commercial Property,industry_Education,industry_Government
3539040,2012-12-31 23:00:00,41.25,UnivLab_Tracy,weather8.csv,Education,5968.7,UnivLab,2012,12,23,...,0,0,0,0,0,0,1,0,1,0
3539041,2013-01-01 00:00:00,41.50,UnivLab_Tracy,weather8.csv,Education,5968.7,UnivLab,2013,1,0,...,0,0,0,0,0,0,1,0,1,0
3539042,2013-01-01 01:00:00,41.25,UnivLab_Tracy,weather8.csv,Education,5968.7,UnivLab,2013,1,1,...,0,0,0,0,0,0,1,0,1,0
3539043,2013-01-01 02:00:00,41.00,UnivLab_Tracy,weather8.csv,Education,5968.7,UnivLab,2013,1,2,...,0,0,0,0,0,0,1,0,1,0
3539044,2013-01-01 03:00:00,41.75,UnivLab_Tracy,weather8.csv,Education,5968.7,UnivLab,2013,1,3,...,0,0,0,0,0,0,1,0,1,0


In [66]:
pdTrainY.to_csv('pdTrainY.csv')

In [77]:
pdTestX.to_csv('pdTestX.csv')

In [84]:
pdTestY.to_csv('pdTestY.csv')

In [85]:
a = pd.read_csv('pdTestY.csv')

In [ ]:
def norm_area(pdTrainX):
    for building in pdTrainX['building_name'].unique():
        area= pdTrainX[pdTrainX['building_name']==building]['area']
        area = area.values
        area = area/area.max()
        pdTrainX[pdTrainX['building_name']==building]['area'] = area
    return pdTrainX

In [68]:
pdTrainXNorm = norm_area(pdTrainX)

/Users/kategroschner/anaconda/lib/python3.6/site-packages/ipykernel/__main__.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [69]:
pdTrainXNorm.head()

,timestamp,electricity,building_name,weather_file,industry,area,primary_space_usage,year,month,hour,...,weekday_5,weekday_6,primary_space_usage_Office,primary_space_usage_PrimClass,primary_space_usage_UnivClass,primary_space_usage_UnivDorm,primary_space_usage_UnivLab,industry_Commercial Property,industry_Education,industry_Government
273,2015-01-12 16:00:00,73.49,Office_Abbey,weather0.csv,Education,6891.54454,Office,2015,1,16,...,0,0,1,0,0,0,0,0,1,0
274,2015-01-12 17:00:00,64.77,Office_Abbey,weather0.csv,Education,6891.54454,Office,2015,1,17,...,0,0,1,0,0,0,0,0,1,0
275,2015-01-12 18:00:00,67.43,Office_Abbey,weather0.csv,Education,6891.54454,Office,2015,1,18,...,0,0,1,0,0,0,0,0,1,0
276,2015-01-12 19:00:00,63.57,Office_Abbey,weather0.csv,Education,6891.54454,Office,2015,1,19,...,0,0,1,0,0,0,0,0,1,0
277,2015-01-12 20:00:00,65.10,Office_Abbey,weather0.csv,Education,6891.54454,Office,2015,1,20,...,0,0,1,0,0,0,0,0,1,0
